# APK310

examine and pickle data for APK310

In [1]:
from datetime import datetime, timezone
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import pytz

In [2]:
raw_well_data = '../data/raw_data/EIM-data-APK310/GroundwaterTimeSeriesResults_2023Nov02_190488.csv'

final_pickle = '../data/pickled_data/APK310_all_data.pkl'
short_pickle = '../data/pickled_data/APK310_short.pkl'

well_level_pickle = '../data/pickled_data/APK310_well_data.pkl'

In [3]:
## Load raw data
level_data = pd.read_csv(raw_well_data,
                         low_memory=False)
display(level_data)

,Study_ID,Study_Name,Location_ID,Study_Specific_Location_ID,Location_Name,Well_Tag_ID,Instrument_ID,Field_Collection_Type,Field_Collector,Time_Zone,...,Result_Comment,Study_ID_Alias,Calculated_Latitude_Decimal_Degrees_NAD83HARN,Calculated_Longitude_Decimal_Degrees_NAD83HARN,Calculated_Land_Surface_Elevation_NAVD88_FT,Calculated_Water_Level_Elevation_NAVD88_FT,Calculated_Water_Level_Elevation_Accuracy_FT,Calculated_Water_Level_Depth_Below_Land_Surface_FT,Record_Created_On,Continuous_Result_System_ID
0,EROGWDB,Ecology Eastern Regional Office Groundwater Da...,APK310,APK310,East Valley High School,APK310,71424,Measurement,WA Dept of Ecology,PDT - Pacific Daylight Time (GMT-7),...,NaN,NaN,47.70066,-117.1954,2061.51,1947.49,+/- 10 ft (3 m),114.02,9/24/2018 4:37:06 PM,115062371
1,EROGWDB,Ecology Eastern Regional Office Groundwater Da...,APK310,APK310,East Valley High School,APK310,71424,Measurement,WA Dept of Ecology,PDT - Pacific Daylight Time (GMT-7),...,NaN,NaN,47.70066,-117.1954,2061.51,NaN,+/- 10 ft (3 m),NaN,9/24/2018 4:37:06 PM,115163345
2,EROGWDB,Ecology Eastern Regional Office Groundwater Da...,APK310,APK310,East Valley High School,APK310,71424,Measurement,WA Dept of Ecology,PDT - Pacific Daylight Time (GMT-7),...,NaN,NaN,47.70066,-117.1954,2061.51,1947.49,+/- 10 ft (3 m),114.02,9/24/2018 4:37:06 PM,115062372
3,EROGWDB,Ecology Eastern Regional Office Groundwater Da...,APK310,APK310,East Valley High School,APK310,71424,Measurement,WA Dept of Ecology,PDT - Pacific Daylight Time (GMT-7),...,NaN,NaN,47.70066,-117.1954,2061.51,NaN,+/- 10 ft (3 m),NaN,9/24/2018 4:37:06 PM,115163346
4,EROGWDB,Ecology Eastern Regional Office Groundwater Da...,APK310,APK310,East Valley High School,APK310,71424,Measurement,WA Dept of Ecology,PDT - Pacific Daylight Time (GMT-7),...,NaN,NaN,47.70066,-117.1954,2061.51,1947.48,+/- 10 ft (3 m),114.03,9/24/2018 4:37:06 PM,115062373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190483,EROGWDB,Ecology Eastern Regional Office Groundwater Da...,APK310,APK310,East Valley High School,APK310,71424,Measurement,WA Dept of Ecology,PDT - Pacific Daylight Time (GMT-7),...,NaN,NaN,47.70066,-117.1954,2061.51,NaN,+/- 10 ft (3 m),NaN,9/24/2018 4:37:06 PM,115249911
190484,EROGWDB,Ecology Eastern Regional Office Groundwater Da...,APK310,APK310,East Valley High School,APK310,71424,Measurement,WA Dept of Ecology,PDT - Pacific Daylight Time (GMT-7),...,NaN,NaN,47.70066,-117.1954,2061.51,1948.93,+/- 10 ft (3 m),112.58,9/24/2018 4:37:06 PM,115153466
190485,EROGWDB,Ecology Eastern Regional Office Groundwater Da...,APK310,APK310,East Valley High School,APK310,71424,Measurement,WA Dept of Ecology,PDT - Pacific Daylight Time (GMT-7),...,NaN,NaN,47.70066,-117.1954,2061.51,NaN,+/- 10 ft (3 m),NaN,9/24/2018 4:37:06 PM,115249912
190486,EROGWDB,Ecology Eastern Regional Office Groundwater Da...,APK310,APK310,East Valley High School,APK310,71424,Measurement,WA Dept of Ecology,PDT - Pacific Daylight Time (GMT-7),...,NaN,NaN,47.70066,-117.1954,2061.51,1948.93,+/- 10 ft (3 m),112.58,9/24/2018 4:37:06 PM,115153467


In [4]:
level_data.nunique()

Study_ID                                                   1
Study_Name                                                 1
Location_ID                                                1
Study_Specific_Location_ID                                 1
Location_Name                                              1
Well_Tag_ID                                                1
Instrument_ID                                              1
Field_Collection_Type                                      1
Field_Collector                                            1
Time_Zone                                                  1
Field_Collection_Date                                   3971
Field_Collection_Time                                     24
Field_Collection_Date_Time                             95244
Field_Collection_Reference_Point                           0
Field_Collection_Upper_Depth                               0
Field_Collection_Lower_Depth                               0
Field_Collection_Depth_U

In [5]:
display(level_data['Result_Parameter_Name'])
level_data['Result_Parameter_Name'][31935]

0         Water level in well (depth below measuring point)
1                                        Temperature, water
2         Water level in well (depth below measuring point)
3                                        Temperature, water
4         Water level in well (depth below measuring point)
                                ...                        
190483                                   Temperature, water
190484    Water level in well (depth below measuring point)
190485                                   Temperature, water
190486    Water level in well (depth below measuring point)
190487                                   Temperature, water
Name: Result_Parameter_Name, Length: 190488, dtype: object

'Temperature, water'

In [6]:
level_data['Result_Parameter_Name'].unique()

array(['Water level in well (depth below measuring point)',
       'Temperature, water'], dtype=object)

In [7]:
## Restrict the data to rows where Result_Parameter_Name=='Water level in well (depth below measuring point)'
level_data = level_data.loc[level_data['Result_Parameter_Name']=='Water level in well (depth below measuring point)']
level_data = level_data.rename(columns={'Result_Value':'well_depth'})

In [8]:
level_data[['Field_Collection_Date_Time','well_depth']].sort_values(by='Field_Collection_Date_Time')

,Field_Collection_Date_Time,well_depth
20,1/1/2007 10:00:00 AM,114.04
44,1/1/2007 10:00:00 PM,114.07
22,1/1/2007 11:00:00 AM,114.05
46,1/1/2007 11:00:00 PM,114.05
0,1/1/2007 12:00:00 AM,114.02
...,...,...
130830,9/9/2016 7:00:00 PM,117.95
130808,9/9/2016 8:00:00 AM,118.07
130832,9/9/2016 8:00:00 PM,117.95
130810,9/9/2016 9:00:00 AM,118.05


In [9]:
## Restrict our attention to the Field_Collection_Date_Time, Result_Value, and Time_Zone columns
level_data = level_data[['Field_Collection_Date_Time','well_depth','Time_Zone']]

## Construct a localized timestamp for each measurment and store it in a datetime_recorded column
tz_dict = {'PDT - Pacific Daylight Time (GMT-7)':'Etc/GMT-7', 
           'PST - Pacific Standard Time (GMT-8)':'Etc/GMT-8'}

level_data['Time_Zone']=level_data['Time_Zone'].apply(lambda x: tz_dict[x])

level_data['Field_Collection_Date_Time'] = pd.to_datetime(
    level_data['Field_Collection_Date_Time'], format = '%m/%d/%Y %H:%M:%S %p', utc=False)

times = level_data.Field_Collection_Date_Time.values
zones = level_data.Time_Zone.values
localized_times = []
for time, zone in zip(times, zones):
    localized_times.append(pd.Timestamp(time).tz_localize(zone))

level_data['datetime_recorded'] = localized_times

## Sort by the timestamps
level_data = level_data.sort_values('datetime_recorded')
level_data = level_data.reset_index(drop=True)

## Extract the date to a column
level_data['date'] = level_data['datetime_recorded'].dt.date

## Group measurments recorded on the same date, and compute their average as the daily avg_well_depth
level_data['avg_well_depth'] = level_data.groupby('date')['well_depth'].transform('mean')

## Gather the columns we want, in the order we want
level_data = level_data.drop_duplicates('date')[['date','avg_well_depth']]
level_data = level_data.sort_values(by='date').reset_index(drop=True)

In [10]:
level_data

,date,avg_well_depth
0,2006-06-21,112.190000
1,2006-06-22,112.335833
2,2006-06-23,112.509130
3,2006-06-24,112.715417
4,2006-06-25,112.935000
...,...,...
3966,2017-04-30,103.501250
3967,2017-05-01,103.552500
3968,2017-05-02,103.610833
3969,2017-05-03,103.682500


In [12]:
level_data.to_pickle(well_level_pickle)

## Surface water and weather data


Marcos already pickled the relevant dataframes so I can just read them in and then merge

In [13]:
sw_data_gage_ht = pd.read_pickle('../data/pickled_data/sw_data_gage_ht.pkl')
sw_data_discharge_cfs = pd.read_pickle('../data/pickled_data/sw_data_discharge_cfs.pkl')
noaa_data = pd.read_pickle('../data/pickled_data/noaa_data.pkl')
wx_data = pd.read_pickle('../data/pickled_data/wx_data.pkl')

In [14]:
all_data = level_data.merge(sw_data_gage_ht, how='outer', on='date')
all_data = all_data.merge(sw_data_discharge_cfs, how='outer', on='date')
all_data = all_data.merge(noaa_data, how='outer', on='date')
all_data = all_data.merge(wx_data, how='outer', on='date')
all_data = all_data.sort_values('date')

## Make the index `nice`:
all_data = all_data.reset_index(drop=True)

In [15]:
all_data

,date,avg_well_depth,gage_ht,discharge_cfs,prcp,temp_avg,temp_max,temp_min,hPa_avg,hum_avg,hum_max,hum_min,wind_avg,wind_max,wind_min,gust_avg,gust_max,gust_min
0,1900-10-21,NaN,NaN,2410.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1900-10-22,NaN,NaN,2750.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1900-10-23,NaN,NaN,3100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1900-10-24,NaN,NaN,3280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1900-10-25,NaN,NaN,3460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44921,2023-10-17,NaN,17.94,1470.0,NaN,53.655000,61.41,43.72,1020.958333,61.208333,80.0,40.0,9.831667,18.41,0.00,0.0,0.0,0.0
44922,2023-10-18,NaN,17.95,1480.0,NaN,55.562083,72.59,42.53,1021.166667,58.708333,83.0,28.0,5.322917,10.36,0.00,0.0,0.0,0.0
44923,2023-10-19,NaN,17.95,1480.0,NaN,57.589167,76.69,42.44,1017.958333,62.375000,87.0,34.0,7.910417,16.11,0.00,0.0,0.0,0.0
44924,2023-10-20,NaN,17.94,1470.0,NaN,59.808750,75.16,46.81,1015.375000,61.458333,76.0,40.0,6.279583,8.05,3.44,0.0,0.0,0.0


In [16]:
# a date to datetime fix

## Make the dates `nice`:
## check the first and last date from above and make sure the number of
## days matches so there are no missing days
date_rng = pd.date_range(start='1900-10-21', end='2023-10-21', freq='D')
all_data['date'] = date_rng

In [17]:
# truncating the data to the relevant dates
# this may depend on the well, but I'm relatively sure that
# these dates are included for all of our wells of interest
min_date = datetime(2006,6,21)
max_date = datetime(2017,9,28)
df_short = all_data.loc[(all_data.date >= min_date) & (all_data.date <= max_date)].copy()

In [18]:
## pickle the full data and the short version
all_data.to_pickle(final_pickle)
df_short.to_pickle(short_pickle)